## dataset generation

In [ ]:
# 基于所有流程条目生成多样化问法
process_templates = [
    ("如何办理{流程}？需要哪些材料？", 0),
    ("{流程}的具体步骤是什么？", 0),
    ("办理{流程}要去哪个办公室？", 0),
    ("申请{流程}的截止日期是多久？", 0),
    ("如果{流程}被驳回怎么办？", 0),
    ("{流程}的审批需要几天？", 0),
    ("网上能办理{流程}吗？", 0)
]

process_data = [
    "云南大学领取本科学历学位证书流程",
    "大学生创新创业训练项目工作流程",
    "本科学生毕业论文（设计）工作流程",
    "实践教学科项目经费报销流程",
    "本科学生放弃辅修或辅修类型变更办理流程",
    "学生申请课程免修办理流程",
    "学生申请课程免听办理流程",
    "学生申请课程缓考办理流程",
    "学生毕业审核申请学分认定办理流程",
    "申请学历或学位证明材料办理流程",
    "申请修改学信网信息办理流程",
    "教师办理成绩录入延期、成绩录入系数更改流程",
    "成绩单、学籍证明打印办理流程",
    "毕业证、学位证遗失补办证明书流程",
    "毕业证、学位证补换发流程",
    "调停课申请流程",
    "临时使用教室申请流程",
    "补办学生证流程"
]

# 实体属性查询模板
teacher_questions = [
    ("{教师}的研究方向是什么？", 1),
    ("怎么联系{教师}老师？", 1),
    ("{教师}教授最近发表了哪些论文？", 1),
    ("{教师}的办公室在哪个校区？", 1)
    ("介绍以下{教师}", 1)
]

course_questions = [
    ("{课程}的上课时间是什么时候？", 1),
    ("{课程}在哪个教室上课？", 1),
    ("{课程}的学分是多少？", 1),
    ("{课程}的授课教师是谁？", 1)
]

classroom_questions = [
    ("{教室}今天有什么课？", 1),
    ("{教室}能容纳多少人考试？", 1),
    ("{教室}属于哪个教学楼？", 1)
]

# 实体数据注入


teachers = ["华宇", "林玲", "胡茂", "池宗琳","闵文文", "张学杰","岳昆","普园媛","徐丹","钱文华","梁洁","张榆锋","张任远","陶大鹏","李浩","赵征鹏","王津","袁洪","张俊华","周丽华","梁虹","王顺芳","聂仁灿","官铮","赵春娜","杨晶晶","武浩","李鹏","袁国武","诸薇娜","周小兵","李波","李海燕","柏正尧"	]  
courses = ["管理心理学", "C语言程序设计", "APP的设计与开发入门", "情绪管理", "求职解码-大学生就业指导","企业经营模拟","商务礼仪","花卉鉴赏与栽培","创业法律风险防范","网络伦理与法制"]  # 完整课程列表
classrooms = ["格物楼2栋2103", "文汇楼2栋2413", "文渊楼310","文渊楼211","文渊楼212","文渊楼213","力行楼4108（小阶梯教室）","力行楼4107（小阶梯教室）","力行楼4207（机房）","力行楼1201（大阶梯教室）","软件学院1327","软件学院1431"]  # 完整教室列表

complex_templates = [
    ("{教师}教授在{时间}有哪些课？", 1),  # 需结合Meeting节点
    ("{课程}的{周次}上课地点在哪？", 1),  # 需关联WeekRange
    ("{教学楼}本周有哪些创新创业课程？", 1)  # 需跨CourseSession和CourseCategory
]

other_questions = [
    ("今天呈贡校区的天气如何？", 2),
    ("讲一个关于大学生活的笑话", 2)
]
#同义词替换引擎
synonym_map = {
    "办理": ["申请", "处理", "操作"],
    "课程": ["课", "科目", "教学班"],
    "流程": ["手续", "步骤", "程序"]
}

def augment_question(text):
    for word, synonyms in synonym_map.items():
        if word in text:
            replaced = random.choice(synonyms)
            return text.replace(word, replaced)
    return text

#句式变异生成
sentence_structures = [
    "请问{query}",
    "同学知道{query}吗",
    "麻烦告诉我{query}",
    "{query}，谢谢！"
]

#模糊表达模拟
fuzzy_patterns = [
    ("{教室}有什么课", "哪个教室在{时间}有空"),
    ("{教师}的论文", "{教师}发表过什么文章")
]

In [22]:
import random
from faker import Faker
import csv
import jieba

# ======== 初始化配置 ========
fake = Faker(locale='zh_CN')
random.seed(42)  # 确保可重复性

# ======== 数据模板 ========
process_templates = [
    ("如何办理{流程}？需要哪些材料？", 0),
    ("{流程}的具体步骤是什么？", 0),
    ("办理{流程}要去哪个办公室？", 0),
    ("申请{流程}的截止日期是多久？", 0),
    ("如果{流程}被驳回怎么办？", 0),
    ("{流程}的审批需要几天？", 0),
    ("网上能办理{流程}吗？", 0),
    ("如何申请{流程}？", 0)
]

teacher_questions = [
    ("{教师}的研究方向是什么？", 1),
    ("怎么联系{教师}老师？", 1),
    ("{教师}教授最近发表了哪些论文？", 1),
    ("{教师}的办公室在哪个校区？", 1),
    ("介绍一下{教师}", 1),
    ("{教师}这学期教什么课？", 1),
    ("{教师}有哪些著作？", 1),
]

course_questions = [
    ("{课程}的上课时间是什么时候？", 1),
    ("{课程}在哪个教室上课？", 1),
    ("{课程}的学分是多少？", 1),
    ("{课程}的授课教师是谁？", 1)
]

classroom_questions = [
    ("{教室}今天有什么课？", 1),
    ("{教室}能容纳多少人考试？", 1),
    ("{教室}属于哪个教学楼？", 1)
]

complex_templates = [
    ("{教师}教授在{时间}有哪些课？", 1),
    ("{课程}的{周次}上课地点在哪？", 1),
    ("{教学楼}本周有哪些创新创业课程？", 1)
]

# ======== 实体数据 ========
process_data = [
    "云南大学领取本科学历学位证书流程",
    "大学生创新创业训练项目工作流程",
    "本科学生毕业论文（设计）工作流程",
    "实践教学科项目经费报销流程",
    "本科学生放弃辅修或辅修类型变更办理流程",
    "学生申请课程免修办理流程",
    "学生申请课程免听办理流程",
    "学生申请课程缓考办理流程",
    "学生毕业审核申请学分认定办理流程",
    "申请学历或学位证明材料办理流程",
    "申请修改学信网信息办理流程",
    "教师办理成绩录入延期、成绩录入系数更改流程",
    "成绩单、学籍证明打印办理流程",
    "毕业证、学位证遗失补办证明书流程",
    "毕业证、学位证补换发流程",
    "调停课申请流程",
    "临时使用教室申请流程",
    "补办学生证流程"
]

teachers = ["华宇", "林玲", "胡茂", "池宗琳","闵文文", "张学杰","岳昆","普园媛","徐丹","钱文华","梁洁","张榆锋","张任远","陶大鹏","李浩","赵征鹏","王津","袁洪","张俊华","周丽华","梁虹","王顺芳","聂仁灿","官铮","赵春娜","杨晶晶","武浩","李鹏","袁国武","诸薇娜","周小兵","李波","李海燕","柏正尧"	]  
courses = ["管理心理学", "C语言程序设计", "APP的设计与开发入门", "情绪管理", "求职解码-大学生就业指导","企业经营模拟","商务礼仪","花卉鉴赏与栽培","创业法律风险防范","网络伦理与法制"]  # 完整课程列表
classrooms = ["格物楼2栋2103", "文汇楼2栋2413", "文渊楼310","文渊楼211","文渊楼212","文渊楼213","力行楼4108（小阶梯教室）","力行楼4107（小阶梯教室）","力行楼4207（机房）","力行楼1201（大阶梯教室）","软件学院1327","软件学院1431"]  # 完整教室列表


# ======== 增强配置 ========
synonym_map = {
    "办理": ["申请", "处理", "操作"],
    "课程": ["课", "科目", "教学班"],
    "流程": ["手续", "步骤", "程序"]
}

sentence_structures = [
    "请问{query}",
    "同学知道{query}吗",
    "麻烦告诉我{query}",
    "{query}，谢谢！"
]

other_questions = [
    ("今天呈贡校区的天气如何？", 2),
    ("讲一个关于大学生活的笑话", 2),
    ("帮我推荐云南大学附近的奶茶店", 2),
    ("123456 是什么", 2),
    ("嗷呜~", 2),
    ("你喜欢吃什么？", 2),
    ("讲一个笑话", 2),
    ("你会做饭吗？", 2),
    ("你喜欢什么颜色？", 2),
    ("你最喜欢的电影是什么？", 2),
    ("你觉得云南大学怎么样？", 2),
    ("你喜欢运动吗？", 2),
    ("你会说英语吗？", 2),
    ("你喜欢听什么音乐？", 2),
    ("你觉得生活中最重要的是什么？", 2),
    ("你喜欢吃什么？", 2),
   ("昆明有哪些好玩的地方？", 2),
   ("大学生如何平衡学习和生活？", 2),
   ("推荐几本适合大学生读的书", 2),
   ("如何提高英语口语能力？", 2),
   ("云南的特色美食有哪些？", 2),
   ("大学生兼职需要注意什么？", 2),
   ("如何申请奖学金？", 2),
   ("昆明有哪些著名的旅游景点？", 2),
   ("大学生如何规划自己的职业发展？", 2),
   ("推荐几部好看的电影", 2),
   ("如何高效地学习编程？", 2),
   ("云南的民族风情有哪些？", 2),
   ("大学生如何保持良好的心理健康？", 2),
   ("推荐几款好用的学习APP", 2),
   ("如何写一篇优秀的论文？", 2),
   ("昆明有哪些好吃的火锅店？", 2),
   ("大学生如何培养自己的兴趣爱好？", 2),
   ("推荐几首好听的歌曲", 2),
   ("如何提高自己的沟通能力？", 2),
   ("云南的气候特点是什么？", 2),
   ("大学生如何管理自己的时间？", 2),
   ("推荐几款好用的笔记软件", 2),
   ("如何准备公务员考试？", 2),
   ("昆明有哪些值得一去的博物馆？", 2),
   ("大学生如何提升自己的综合素质？", 2),
   ("推荐几款好用的健身APP", 2),
   ("如何提高自己的写作能力？", 2),
   ("云南的旅游景点推荐", 2),
   ("大学生如何处理人际关系？", 2),
   ("推荐几款好用的英语学习APP", 2),
   ("如何准备考研？", 2),
   ("昆明有哪些好吃的烧烤店？", 2),
   ("大学生如何培养自己的领导力？", 2),
   ("推荐几款好用的阅读APP", 2),
   ("如何提高自己的演讲能力？", 2),
   ("云南的少数民族有哪些？", 2),
   ("大学生如何规划自己的假期？", 2),
   ("推荐几款好用的理财APP", 2),
   ("如何提高自己的逻辑思维能力？", 2),
   ("昆明有哪些好吃的过桥米线店？", 2),
   ("大学生如何培养自己的创新思维？", 2),
   ("推荐几款好用的背单词APP", 2),
   ("如何提高自己的团队合作能力？", 2),
   ("云南的特产有哪些？", 2),
   ("大学生如何管理自己的财务？", 2),
   ("推荐几款好用的日记APP", 2),
   ("如何提高自己的抗压能力？", 2),
   ("昆明有哪些好吃的甜品店？", 2),
   ("大学生如何培养自己的自律能力？", 2),
   ("推荐几款好用的翻译APP", 2),
   ("如何提高自己的面试技巧？", 2),
   ("云南的传统文化有哪些？", 2),
   ("大学生如何规划自己的未来？", 2),
   ("推荐几款好用的任务管理APP", 2),
   ("如何提高自己的学习能力？", 2),
   ("昆明有哪些好吃的早餐店？", 2),
   ("大学生如何培养自己的独立思考能力？", 2),
   ("推荐几款好用的音乐播放APP", 2),
   ("如何提高自己的社交能力？", 2),
   ("云南的历史文化有哪些？", 2),
   ("大学生如何规划自己的学业？", 2),
   ("推荐几款好用的视频学习APP", 2),
   ("如何提高自己的创新能力？", 2),
   ("昆明有哪些好吃的素食餐厅？", 2),
   ("大学生如何培养自己的责任感？", 2),
   ("推荐几款好用的健身软件", 2),
   ("如何提高自己的表达能力？", 2),
   ("云南的自然风光有哪些？", 2),
   ("大学生如何规划自己的职业规划？", 2),
   ("推荐几款好用的阅读软件", 2),
   ("如何提高自己的组织能力？", 2),
   ("昆明有哪些好吃的海鲜餐厅？", 2),
   ("大学生如何培养自己的自信心？", 2),
   ("推荐几款好用的英语学习软件", 2),
   ("如何提高自己的决策能力？", 2),
   ("云南的民俗文化有哪些？", 2),
   ("大学生如何规划自己的时间管理？", 2),
   ("推荐几款好用的笔记软件", 2),
   ("如何提高自己的沟通技巧？", 2),
   ("昆明有哪些好吃的川菜馆？", 2),
   ("大学生如何培养自己的团队合作精神？", 2),
   ("推荐几款好用的学习软件", 2),
   ("如何提高自己的写作技巧？", 2),
   ("云南的旅游景点有哪些？", 2),
   ("大学生如何处理恋爱关系？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的演讲技巧？", 2),
   ("昆明有哪些好吃的粤菜馆？", 2),
   ("大学生如何培养自己的领导能力？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的逻辑思维？", 2),
   ("云南的少数民族文化有哪些？", 2),
   ("大学生如何规划自己的假期生活？", 2),
   ("推荐几款好用的理财工具", 2),
   ("如何提高自己的团队合作技巧？", 2),
   ("昆明有哪些好吃的湘菜馆？", 2),
   ("大学生如何培养自己的创新能力？", 2),
   ("推荐几款好用的背单词工具", 2),
   ("如何提高自己的抗压技巧？", 2),
   ("云南的特产文化有哪些？", 2),
   ("大学生如何管理自己的财务状况？", 2),
   ("推荐几款好用的日记工具", 2),
   ("如何提高自己的面试能力？", 2),
   ("昆明有哪些好吃的东北菜馆？", 2),
   ("大学生如何培养自己的自律精神？", 2),
   ("推荐几款好用的翻译工具", 2),
   ("如何提高自己的学习能力？", 2),
   ("云南的传统文化有哪些特色？", 2),
   ("大学生如何规划自己的未来职业？", 2),
   ("推荐几款好用的任务管理工具", 2),
   ("如何提高自己的社交技巧？", 2),
   ("昆明有哪些好吃的西北菜馆？", 2),
   ("大学生如何培养自己的独立思考精神？", 2),
   ("推荐几款好用的音乐播放工具", 2),
   ("如何提高自己的创新能力？", 2),
   ("云南的历史文化有哪些特色？", 2),
   ("大学生如何规划自己的学业生涯？", 2),
   ("推荐几款好用的视频学习工具", 2),
   ("如何提高自己的表达能力？", 2),
   ("昆明有哪些好吃的东南亚菜馆？", 2),
   ("大学生如何培养自己的责任感？", 2),
   ("推荐几款好用的健身工具", 2),
   ("如何提高自己的组织能力？", 2),
   ("云南的自然风光有哪些特色？", 2),
   ("大学生如何规划自己的职业生涯？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的决策能力？", 2),
   ("昆明有哪些好吃的日料餐厅？", 2),
   ("大学生如何培养自己的自信心？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的沟通能力？", 2),
   ("云南的民俗文化有哪些特色？", 2),
   ("大学生如何规划自己的时间安排？", 2),
   ("推荐几款好用的笔记工具", 2),
   ("如何提高自己的写作能力？", 2),
   ("昆明有哪些好吃的韩料餐厅？", 2),
   ("大学生如何培养自己的团队合作精神？", 2),
   ("推荐几款好用的学习工具", 2),
   ("如何提高自己的演讲能力？", 2),
   ("云南的旅游景点有哪些特色？", 2),
   ("大学生如何处理恋爱关系？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的逻辑思维能力？", 2),
   ("昆明有哪些好吃的法式餐厅？", 2),
   ("大学生如何培养自己的领导能力？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的团队合作能力？", 2),
   ("云南的少数民族文化有哪些特色？", 2),
   ("大学生如何规划自己的假期安排？", 2),
   ("推荐几款好用的理财工具", 2),
   ("如何提高自己的抗压能力？", 2),
   ("昆明有哪些好吃的意式餐厅？", 2),
   ("大学生如何培养自己的创新能力？", 2),
   ("推荐几款好用的背单词工具", 2),
   ("如何提高自己的面试技巧？", 2),
   ("云南的特产文化有哪些特色？", 2),
   ("大学生如何管理自己的财务状况？", 2),
   ("推荐几款好用的日记工具", 2),
   ("如何提高自己的学习能力？", 2),
   ("昆明有哪些好吃的泰式餐厅？", 2),
   ("大学生如何培养自己的自律精神？", 2),
   ("推荐几款好用的翻译工具", 2),
   ("如何提高自己的社交能力？", 2),
   ("云南的传统文化有哪些特色？", 2),
   ("大学生如何规划自己的未来职业？", 2),
   ("推荐几款好用的任务管理工具", 2),
   ("如何提高自己的表达能力？", 2),
   ("昆明有哪些好吃的印度餐厅？", 2),
   ("大学生如何培养自己的独立思考精神？", 2),
   ("推荐几款好用的音乐播放工具", 2),
   ("如何提高自己的创新能力？", 2),
   ("云南的历史文化有哪些特色？", 2),
   ("大学生如何规划自己的学业生涯？", 2),
   ("推荐几款好用的视频学习工具", 2),
   ("如何提高自己的组织能力？", 2),
   ("昆明有哪些好吃的巴西餐厅？", 2),
   ("大学生如何培养自己的责任感？", 2),
   ("推荐几款好用的健身工具", 2),
   ("如何提高自己的决策能力？", 2),
   ("云南的自然风光有哪些特色？", 2),
   ("大学生如何规划自己的职业生涯？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的沟通技巧？", 2),
   ("昆明有哪些好吃的墨西哥餐厅？", 2),
   ("大学生如何培养自己的自信心？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的写作技巧？", 2),
   ("云南的民俗文化有哪些特色？", 2),
   ("大学生如何规划自己的时间安排？", 2),
   ("推荐几款好用的笔记工具", 2),
   ("如何提高自己的演讲技巧？", 2),
   ("昆明有哪些好吃的德国餐厅？", 2),
   ("大学生如何培养自己的团队合作精神？", 2),
   ("推荐几款好用的学习工具", 2),
   ("如何提高自己的逻辑思维？", 2),
   ("云南的旅游景点有哪些特色？", 2),
   ("大学生如何处理恋爱关系？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的团队合作技巧？", 2),
   ("昆明有哪些好吃的比利时餐厅？", 2),
   ("大学生如何培养自己的领导能力？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的抗压技巧？", 2),
   ("云南的少数民族文化有哪些特色？", 2),
   ("大学生如何规划自己的假期安排？", 2),
   ("推荐几款好用的理财工具", 2),
   ("如何提高自己的面试能力？", 2),
   ("昆明有哪些好吃的西班牙餐厅？", 2),
   ("大学生如何培养自己的创新能力？", 2),
   ("推荐几款好用的背单词工具", 2),
   ("如何提高自己的学习能力？", 2),
   ("云南的特产文化有哪些特色？", 2),
   ("大学生如何管理自己的财务状况？", 2),
   ("推荐几款好用的日记工具", 2),
   ("如何提高自己的社交技巧？", 2),
   ("昆明有哪些好吃的葡萄牙餐厅？", 2),
   ("大学生如何培养自己的自律精神？", 2),
   ("推荐几款好用的翻译工具", 2),
   ("如何提高自己的表达能力？", 2),
   ("云南的传统文化有哪些特色？", 2),
   ("大学生如何规划自己的未来职业？", 2),
   ("推荐几款好用的任务管理工具", 2),
   ("如何提高自己的创新能力？", 2),
   ("昆明有哪些好吃的俄罗斯餐厅？", 2),
   ("大学生如何培养自己的独立思考精神？", 2),
   ("推荐几款好用的音乐播放工具", 2),
   ("如何提高自己的组织能力？", 2),
   ("云南的历史文化有哪些特色？", 2),
   ("大学生如何规划自己的学业生涯？", 2),
   ("推荐几款好用的视频学习工具", 2),
   ("如何提高自己的决策能力？", 2),
   ("昆明有哪些好吃的希腊餐厅？", 2),
   ("大学生如何培养自己的责任感？", 2),
   ("推荐几款好用的健身工具", 2),
   ("如何提高自己的沟通能力？", 2),
   ("云南的自然风光有哪些特色？", 2),
   ("大学生如何规划自己的职业生涯？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的写作能力？", 2),
   ("昆明有哪些好吃的土耳其餐厅？", 2),
   ("大学生如何培养自己的自信心？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的演讲能力？", 2),
   ("云南的民俗文化有哪些特色？", 2),
   ("大学生如何规划自己的时间安排？", 2),
   ("推荐几款好用的笔记工具", 2),
   ("如何提高自己的逻辑思维能力？", 2),
   ("昆明有哪些好吃的阿根廷餐厅？", 2),
   ("大学生如何培养自己的团队合作精神？", 2),
   ("推荐几款好用的学习工具", 2),
   ("如何提高自己的团队合作能力？", 2),
   ("云南的旅游景点有哪些特色？", 2),
   ("大学生如何处理恋爱关系？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的抗压能力？", 2),
   ("昆明有哪些好吃的澳大利亚餐厅？", 2),
   ("大学生如何培养自己的领导能力？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的面试技巧？", 2),
   ("云南的少数民族文化有哪些特色？", 2),
   ("大学生如何规划自己的假期安排？", 2),
   ("推荐几款好用的理财工具", 2),
   ("如何提高自己的学习能力？", 2),
   ("昆明有哪些好吃的南非餐厅？", 2),
   ("大学生如何培养自己的创新能力？", 2),
   ("推荐几款好用的背单词工具", 2),
   ("如何提高自己的社交能力？", 2),
   ("云南的特产文化有哪些特色？", 2),
   ("大学生如何管理自己的财务状况？", 2),
   ("推荐几款好用的日记工具", 2),
   ("如何提高自己的表达能力？", 2),
   ("昆明有哪些好吃的埃及餐厅？", 2),
   ("大学生如何培养自己的自律精神？", 2),
   ("推荐几款好用的翻译工具", 2),
   ("如何提高自己的创新能力？", 2),
   ("云南的传统文化有哪些特色？", 2),
   ("大学生如何规划自己的未来职业？", 2),
   ("推荐几款好用的任务管理工具", 2),
   ("如何提高自己的组织能力？", 2),
   ("昆明有哪些好吃的加拿大餐厅？", 2),
   ("大学生如何培养自己的独立思考精神？", 2),
   ("推荐几款好用的音乐播放工具", 2),
   ("如何提高自己的决策能力？", 2),
   ("云南的历史文化有哪些特色？", 2),
   ("大学生如何规划自己的学业生涯？", 2),
   ("推荐几款好用的视频学习工具", 2),
   ("如何提高自己的沟通技巧？", 2),
   ("昆明有哪些好吃的新西兰餐厅？", 2),
   ("大学生如何培养自己的责任感？", 2),
   ("推荐几款好用的健身工具", 2),
   ("如何提高自己的写作技巧？", 2),
   ("云南的自然风光有哪些特色？", 2),
   ("大学生如何规划自己的职业生涯？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的演讲技巧？", 2),
   ("昆明有哪些好吃的以色列餐厅？", 2),
   ("大学生如何培养自己的自信心？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的逻辑思维？", 2),
   ("云南的民俗文化有哪些特色？", 2),
   ("大学生如何规划自己的时间安排？", 2),
   ("推荐几款好用的笔记工具", 2),
   ("如何提高自己的团队合作技巧？", 2),
   ("昆明有哪些好吃的伊朗餐厅？", 2),
   ("大学生如何培养自己的团队合作精神？", 2),
   ("推荐几款好用的学习工具", 2),
   ("如何提高自己的抗压技巧？", 2),
   ("云南的旅游景点有哪些特色？", 2),
   ("大学生如何处理恋爱关系？", 2),
   ("推荐几款好用的英语学习工具", 2),
   ("如何提高自己的面试能力？", 2),
   ("昆明有哪些好吃的沙特阿拉伯餐厅？", 2),
   ("大学生如何培养自己的领导能力？", 2),
   ("推荐几款好用的阅读工具", 2),
   ("如何提高自己的学习能力？", 2),
   ("云南的少数民族文化有哪些特色？", 2),
   ("大学生如何规划自己的假期安排？", 2),
   ("推荐几款好用的理财工具", 2),
   ("如何提高自己的社交技巧？", 2),
   ("昆明有哪些好吃的泰国餐厅？", 2),
   ("大学生如何培养自己的创新能力？", 2),
   ("推荐几款好用的背单词工具", 2)]

# ======== 核心生成逻辑 ========
def generate_samples():
    # 流程类问题生成（带分层增强）
    for process in process_data:
        for template, label in process_templates:
            base_question = template.format(流程=process)
            
            # 第一层：基础问题 + 全句式模板
            for sent_tpl in sentence_structures:
                yield (sent_tpl.format(query=base_question), label)
            
            # 第二层：方言变体 + 随机1个句式模板
            dialect_question = base_question.replace("如何", "咋个")
            selected_tpl = random.choice(sentence_structures)
            yield (selected_tpl.format(query=dialect_question), label)
            
            # 第三层：同义词替换 + 随机1个句式模板
            for _ in range(2):
                augmented = augment_question(base_question)
                if augmented != base_question:
                    selected_tpl = random.choice(sentence_structures)
                    yield (selected_tpl.format(query=augmented), label)

    # 教师类问题生成（智能模板匹配）
    for teacher in teachers:
        for tpl, label in teacher_questions:
            base = tpl.format(教师=teacher)
            
            # 根据问题类型选择最佳句式模板
            if "研究方向" in base:
                templates = [sentence_structures[0], sentence_structures[2]]  # 优先正式句式
            else:
                templates = sentence_structures
            
            for sent_tpl in templates:
                yield (sent_tpl.format(query=base), label)
    # 其他类问题生成（不带分层增强）
    for other in other_questions:
            base = other[0]
            label = other[1]
            yield (base, label)
            yield (sentence_structures[0].format(query=base), label)
            yield (sentence_structures[2].format(query=base), label)
    # 时空查询类问题生成（带上下文感知）
    time_units = ["上午", "下午", "晚上"]
    for course in courses:
        # 生成时间相关查询
        base = f"{random.choice(time_units)}的{course}在哪上？"
        yield (random.choice(sentence_structures).format(query=base), 1)
        
        # 生成周次查询（保持原始句式）
        base = f"{course}的{random.choice(['1-5周', '6-10周'])}上课地点"
        yield (base, 1)  # 保留部分原始问题

# ======== 增强函数 ========
def augment_question(text):
    """智能同义词替换"""
    replaced = []
    for word in jieba.lcut(text):
        # 保留核心实体不替换
        if word in teachers + classrooms + courses:
            replaced.append(word)
            continue
            
        if word in synonym_map and random.random() < 0.6:  # 60%替换概率
            replaced.append(random.choice(synonym_map[word]))
        else:
            replaced.append(word)
    return "".join(replaced)

# ======== 数据保存 ========
def save_to_csv(filename, samples, max_count=10000):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['text', 'label'])
        
        count = 0
        for text, label in samples:
            if count >= max_count:
                break
            # 应用句式变异
            if random.random() < 0.8:  # 30%的概率添加句式变异
                template = random.choice(sentence_structures)
                text = template.format(query=text)
                
            writer.writerow([text, label])
            count += 1

# ======== 执行生成 ========
if __name__ == "__main__":
    # 生成样本
    samples = generate_samples()
    print(samples)
    # 保存为CSV
    save_to_csv('training_data.csv', samples, max_count=10000)
    

<generator object generate_samples at 0x000001F3DD5C3E20>


In [23]:
from collections import Counter
with open("./training_data.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    data = list(reader)
print(Counter([label for _, label in data]))


Counter({'2': 1014, '0': 944, '1': 904})


## Train

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import jieba

# ======== 配置参数 ========
MODEL_NAME = "distilbert-base-multilingual-cased"  # 轻量级多语言版
MAX_LENGTH = 64                                    # 文本最大长度
BATCH_SIZE = 32                                    # 根据GPU显存调整
NUM_EPOCHS = 4                                     # 最佳实践值
LABEL_MAP = {"vector_db": 0, "neo4j": 1, "other": 2}

# ======== 数据加载 ========
df = pd.read_csv("./training_data.csv")
texts = df["text"].tolist()
labels = df["label"].tolist()

# 数据预处理（中文特殊处理）
def preprocess_chinese(text):
    return " ".join(jieba.lcut(text))  # 加入分词提升模型理解

texts = [preprocess_chinese(t) for t in texts]

# 数据集划分
X_train, X_temp, y_train, y_temp = train_test_split(texts, labels, test_size=0.2, stratify=labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

# ======== 模型初始化 ========
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=3,
    id2label={v:k for k,v in LABEL_MAP.items()},
    label2id=LABEL_MAP
)

# ======== 数据集封装 ========
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(
            texts, 
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(X_train, y_train)
val_dataset = IntentDataset(X_val, y_val)
test_dataset = IntentDataset(X_test, y_test)

# ======== 评估指标 ========
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds, average="weighted")
    }

# ======== 训练参数 ========
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",          # 参数名修正
    learning_rate=3e-5,
    report_to="none",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="no",
    fp16=True,
    load_best_model_at_end=False    # 添加必要参数
)

# ======== 开始训练 ========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

# ======== 最终评估 ========
final_metrics = trainer.evaluate(test_dataset)
print(f"测试集性能：{final_metrics}")

# ======== 模型保存 ========
model.save_pretrained("./trainresults/")
tokenizer.save_pretrained("./trainresults/")

## Test

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import jieba
import numpy as np

# ======== 配置参数 ========
MODEL_PATH = "./intentionmodel/results1"  # 模型保存目录
LABEL_MAP = {0: "vector_db", 1: "neo4j", 2: "other"}
MAX_LENGTH = 64  # 与训练时保持一致

# ======== 初始化模型和分词器 ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载分词器（需添加领域词汇）
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, local_files_only=True  ).to(device)

# ======== 预处理函数 ========
def preprocess(text):
    """与训练时相同的预处理流程"""
    return " ".join(jieba.lcut(text))

# ======== 预测函数 ========
def predict(text):
    # 预处理
    processed_text = preprocess(text)
    
    # Tokenization
    inputs = tokenizer(
        processed_text,
        max_length=MAX_LENGTH,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).to(device)
    
    # 推理
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 解析结果
    probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    pred_label = np.argmax(probs)
    
    return {
        "text": text,
        "intent": LABEL_MAP[pred_label],
        "confidence": float(probs[pred_label]),
        "details": {LABEL_MAP[i]: float(probs[i]) for i in range(len(LABEL_MAP))}
    }

# ======== 交互测试 ========
if __name__ == "__main__":
    print("输入'q'退出测试")
    while True:
        text = input("\n请输入问题：")
        if text.lower() == 'q':
            break
            
        result = predict(text)
        print(f"\n预测结果：{result['intent']}（置信度：{result['confidence']:.2%}）")
        print("详细概率：")
        for k, v in result["details"].items():
            print(f"  {k}: {v:.2%}")

输入'q'退出测试

预测结果：neo4j（置信度：99.96%）
详细概率：
  vector_db: 0.02%
  neo4j: 99.96%
  other: 0.02%

预测结果：other（置信度：99.80%）
详细概率：
  vector_db: 0.02%
  neo4j: 0.18%
  other: 99.80%


## 添加分词再训练

In [5]:
import json

def generate_jieba_dict():
    with open('./entities/classroom.json', 'r', encoding='utf-8') as f:
        classrooms = json.load(f)
    with open('./entities/course.json', 'r', encoding='utf-8') as f:
        courses = json.load(f)
    with open('./entities/teachers.json', 'r', encoding='utf-8') as f:
        teachers = json.load(f)
    with open('./entities/building.json', 'r', encoding='utf-8') as f:
        buildings = json.load(f)
    with open('./ynudict.txt', 'w', encoding='utf-8') as f:
        for teacher in teachers:
            if teacher['name'] == None:
                continue
            f.write(f"{teacher['name']} 100 nr\n")
        
        for room in classrooms:
            if room['name'] == None:
                continue
            f.write(f"{room['name']} 200 ns\n")

        for building in buildings:
            if building['name'] == None:
                continue
            f.write(f"{building['name']} 200 ns\n")
        
        for course in courses:
            if course["name"] == None:
                continue
            main_name = course["name"].split("-")[0].split("（")[0]
            f.write(f"{main_name} 150 nz\n")
        processes = ["补办学生证", "缓考申请", "教室申请", "毕业证", "学生证", "成绩单", "学籍证明", "学信网信息", '学分认定', '课程缓考', '课程免听', '课程免修', '经费报销']
        for p in processes:
            f.write(f"{p} 300 n\n")
generate_jieba_dict()

In [7]:
import jieba

def test_segmentation():
    test_cases = [
        ("王津老师的研究方向", ["王津", "老师", "的", "研究方向"]),
        ("格物楼2栋2103的课程", ["格物楼2栋2103", "的", "课程"]),
        ("申请缓考需要什么材料", ["申请", "缓考", "需要", "什么", "材料"])
    ]
    
    jieba.load_userdict("./ynudict.txt")
    
    errors = 0
    for text, expected in test_cases:
        actual = jieba.lcut(text)
        if actual != expected:
            print(f"错误：{text} → {actual}")
            errors += 1
    
    print(f"测试完成，错误率：{errors/len(test_cases):.0%}")

test_segmentation()

测试完成，错误率：0%


In [9]:
import pandas as pd
import jieba

# 加载自定义词典（关键步骤！）
jieba.load_userdict("./ynudict.txt")  # 你的词典文件路径

def rebuild_dataset(input_csv, output_csv):
    """重建分词后的训练数据"""
    df = pd.read_csv(input_csv)
    
    # 新版分词函数
    def seg(text):
        return " ".join(jieba.lcut(text))
    
    # 应用新分词
    df["processed_text"] = df["text"].apply(seg)
    
    # 保存新数据集
    df[["processed_text", "label"]].to_csv(output_csv, index=False)
    print(f"已重建数据集保存至 {output_csv}")

# 执行（假设原数据为 training_data.csv）
rebuild_dataset("./training_data.csv", "./retrain_data_v3.csv")

已重建数据集保存至 ./retrain_data_v3.csv
